In [1]:
import numpy as np
import pandas
import string
import urllib
from bs4 import BeautifulSoup as bs
import datetime
import dateutil
from pprint import pprint
import os
import re
from datetime import datetime
print('ok')

ok


In [2]:
# Read dataset
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read')

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

### SUMMARY STATISTICS FOR FILENAME DATASET
print(len(music_df))
# 2,973 filenames

print('\n')
print(
    music_df
    .groupby('source')
    .size()
    .sort_values(ascending=False)
)
# Sources:
#DimeaDozen      1804
#Etree            564
#LosslessLegs     275
#TradersDen       330

print('\n')
print(
    music_df['filename'].
    groupby(music_df['filename'].
            apply(lambda x: x.lower().split('.')[-1])).
    size().
    sort_values(ascending=False)
)
# File extensions:
#flac    2798
#shn       92
#mp2       48
#mp3       31
#wav        1
#cue        1
#aud        1
#FLAC       1
print('ok')

Datafile read
Columns renamed
2973


source
DimeaDozen      1804
Etree            564
TradersDen       330
LosslessLegs     275
dtype: int64


filename
flac    2799
shn       92
mp2       48
mp3       31
wav        1
cue        1
aud        1
Name: filename, dtype: int64
ok


In [ ]:
'''
STEPS IN FLOWCHART:
A. Does filename start with the artist name? (False -> 0)
B. Does the date come next? (False -> 0)
C. Does an artist name come next? (False -> 0)

D. Does the artist have an official abbrev? (True -> E, False -> G)

E. Is (abbrev used correctly) OR (date correctly formatted)? (True -> F , False -> 1)
F. Is (abbrev used correctly) AND (date correctly formatted)? (True -> H , False -> 2)

G. Is the date correctly formatted? (True -> H, False -> 1)

H. Does the date immediately follow artist name, no space between? (False -> 3)

J. Do they include the mics parameter? (True -> K, False -> L)
K. Do they use the mics parameter correctly? (True -> L, False -> 4)

L. Is taper used correctly? (False -> 4)
M. Is sbesource used correctly? (False -> 4)
N. Is filetype used correctly? (False -> 4)
P. Does filetype accurately describe the type of file? (True -> 5, False -> 4)
'''
print('ok')

In [3]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')

# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
#pprint(abbrev_df.values.tolist())
canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()
#canonical_abbrevs

# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)

# Fix wilco by moving it from variant to canonical
canonical_abbrevs.append('wilco')
variant_abbrevs.remove('wilco')

# Strip and lower
canonical_abbrevs = [x.lower().strip() for x in canonical_abbrevs]    
variant_abbrevs = [x.lower().strip() for x in variant_abbrevs]    

print('Parsed {0:,.0f} canonical abbreviations and {1:,.0f} variant abbreviations.'.format(len(canonical_abbrevs), len(variant_abbrevs)))

Retrieving from etree
Thinking...
Parsed 156 canonical abbreviations and 61 variant abbreviations.


In [4]:
debug = 1
markers = re.compile('[\.\-\_\ ]*')
endmarkers = re.compile('[\.\-\_\ ]$')
startswithbadness = '^[\s\_\-]*(dis|cd|track|artist|unknown)[\s\_\-]*'
startswithdigit = re.compile('^[\d]+')
dateformat = re.compile('[1-9]\d*[\s\/\-]*\d+[\s\/\-]*\d+')
scbands = re.compile('(calloway|earth|gregory|bley|smith|jimmy|zeppelin|redford|lizzy|beck|dust|peter gabriel|stones|peppers|sinatra)')


os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})

print('Filenames in dataset: {0:,.0f}'.format(len(music_df)))

# First, strip filename and convert to lowercase.
music_df['filename'] = music_df['filename'].apply(lambda x: x.lower().strip())

# Next, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])


zeros_df = pandas.DataFrame(data=None, columns=music_df.columns)
ones_df = pandas.DataFrame(data=None, columns=music_df.columns)
twos_df = pandas.DataFrame(data=None, columns=music_df.columns)
threes_df = pandas.DataFrame(data=None, columns=music_df.columns)
fours_df = pandas.DataFrame(data=None, columns=music_df.columns)

zeros_df.index.name = music_df.index.name
ones_df.index.name = music_df.index.name
twos_df.index.name = music_df.index.name
threes_df.index.name = music_df.index.name
fours_df.index.name = music_df.index.name

# If there are no numbers in the remainder part of filename, score as zero and remove
print('\nStep 1. Filenames with no numbers in remainder part: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2a. Filenames with track/cd/disc followed by number with optional whitespace: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search(startswithbadness, x) != None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2b. Starts with a number but has no dates and is not a special case band: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['filename'].apply(lambda x: 
               (re.search(startswithdigit, x) != None) & 
               (re.search(dateformat, x) == None) &
               (re.search(scbands, x) == None)                                                
              )])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2c. Remainder has only number: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search('\D', x) == None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))



Filenames in dataset: 2,973

Step 1. Filenames with no numbers in remainder part: score = 0
Filenames remaining in dataset: 2,955

Step 2a. Filenames with track/cd/disc followed by number with optional whitespace: score = 0
Filenames remaining in dataset: 2,735

Step 2b. Starts with a number but has no dates and is not a special case band: score = 0
Filenames remaining in dataset: 1,805

Step 2c. Remainder has only number: score = 0
Filenames remaining in dataset: 1,803


In [5]:

# Parse up to first digit as band name
music_df['band'] = music_df['remainder'].apply(lambda x: x[0:re.search('\d', x).start()])

# Strip and lower band names
music_df['band'] = music_df['band'].apply(lambda x: x.lower().strip())

# Fix U2
music_df.loc[music_df['band'] == 'u', 'band'] = 'u2'

# Parse everything after the first digit for analysis of dates
music_df['after_first_digit'] = music_df['remainder'].apply(lambda x: x[re.search('\d', x).start():len(x)])

music_df.loc[music_df['band'] == 'u2', 
            'after_first_digit'] = music_df['after_first_digit'][
    music_df['band'] == 'u2'].apply(
lambda x: x[re.search('^2[\s\_\-]*', x).end():len(x)])

print('ok')

ok


In [6]:
# Check band names vs. list of canonical abbreviations
music_df['abbrev_type'] = 'xxx'
music_df.loc[music_df['band'].isin(canonical_abbrevs), 'abbrev_type'] = 'canonical'

# Check band names vs. list of variant abbreviations
music_df.loc[music_df['band'].isin(variant_abbrevs), 'abbrev_type'] = 'variant'

# Parse date as yyyy-mm-dd
music_df['full_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce').dropna()
music_df['full_date_mics_etc'] = music_df['after_first_digit'][music_df['full_date'].notnull()].apply(lambda x: x[10:]).dropna()

music_df['full_date_mics_etc'] = music_df['full_date_mics_etc'].dropna()[music_df['full_date_mics_etc'].dropna().apply(lambda x: re.search(markers, x) != None)].apply(lambda x: x[re.search(markers, x).end():len(x)])

# Parse date as yy-mm-dd
music_df['yy_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce').dropna()
music_df['yy_date_mics_etc'] = music_df['after_first_digit'][music_df['yy_date'].notnull()].apply(lambda x: x[8:]).dropna()
markers = re.compile('[\.\-\_\ ]*')
music_df['yy_date_mics_etc'] = music_df['yy_date_mics_etc'].dropna()[music_df['yy_date_mics_etc'].dropna().apply(lambda x: re.search(markers, x) != None)].apply(lambda x: x[re.search(markers, x).end():len(x)])


# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic', 'jg+ms', 'deadco', 'deadandco', 'dc', 'jgms', 'ph']
incorrect += ['dead and company', 'dead.and.co', 'dead.and.co', 'dac', 'dandc']
incorrect += ['deadco-', 'dead&co', 'dead&amp;co', 'dead&amp;company', 'deadandcompany']
incorrect += ['deadco.irvine.', 'deadfeat', 'gov`t mule', 'gm']

# Band names not in abbreviation list
notfound = ['loslobos', 'deanmonkey&amp;thedropouts', 'ttb', 'crb', 'deadandcompany', 'ny', 'gsbg', 'bd']
notfound += ['jefferson starship', 'carlton, larry -', 'the who', 's marriott', 'unknown artist']
notfound += ['tal wilkenfeld', 'ph&amp;tl', 'sb', 'radiohead', 'theclash_', 'primus']

vc = music_df['band'][music_df['band'] != ''].value_counts()
# If a band name appears 4 or fewer times, assume it's a misc band not found in the list.
for x in (vc[(vc<5)].index.sort_values().tolist()):
    if ((x not in canonical_abbrevs) and (x not in variant_abbrevs) and (x not in notfound) and (x not in incorrect)):
        notfound.append(x)
        
notfound += ['u2']

music_df.loc[music_df['band'].isin(incorrect), 'abbrev_type'] = 'incorrect'
music_df.loc[music_df['band'].isin(notfound), 'abbrev_type'] = 'notfound'
music_df.groupby('abbrev_type').size()

print('\nStep 3a. Canonical abbreviation and yyyy-mm-dd date: score = 4')
fours_df = fours_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(fours_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3b. Variant abbreviation and yyyy-mm-dd date: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3c. Incorrect abbreviation and yyyy-mm-dd date: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3d. Band not found in etree list and yyyy-mm-dd date: score = 4')
fours_df = fours_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(fours_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4a. Canonical abbreviation and yy-mm-dd date: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4b. Variant abbreviation and yy-mm-dd date: score = 2')
twos_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4c. Incorrect abbreviation and yy-mm-dd date: score = 2')
twos_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4d. Band not found in etree list and yy-mm-dd date: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))



Step 3a. Canonical abbreviation and yyyy-mm-dd date: score = 4
Filenames remaining in dataset: 1,669

Step 3b. Variant abbreviation and yyyy-mm-dd date: score = 3
Filenames remaining in dataset: 1,648

Step 3c. Incorrect abbreviation and yyyy-mm-dd date: score = 3
Filenames remaining in dataset: 1,475

Step 3d. Band not found in etree list and yyyy-mm-dd date: score = 4
Filenames remaining in dataset: 674

Step 4a. Canonical abbreviation and yy-mm-dd date: score = 3
Filenames remaining in dataset: 577

Step 4b. Variant abbreviation and yy-mm-dd date: score = 2
Filenames remaining in dataset: 575

Step 4c. Incorrect abbreviation and yy-mm-dd date: score = 2
Filenames remaining in dataset: 564

Step 4d. Band not found in etree list and yy-mm-dd date: score = 3
Filenames remaining in dataset: 551


In [7]:
date1 = re.compile('\d+[\-\/\ —]+\d+[\-\/\ —]+\d+')
date2 = re.compile('(\d{8})')
date3 = re.compile('(19|20)*\d{2}_\d+_\d+')
date4 = re.compile('(19|20)\d{2}[-]\D+[-]\d{1,2}')


music_df.loc[music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None), 
             'parsed_date'] = pandas.to_datetime(
    music_df['after_first_digit'].dropna().loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None)].apply(
lambda x: x[re.search(date1,x).start():re.search(date1,x).end()]), 
                   errors='coerce').dropna()

music_df.loc[music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None), 
             'parsed_date_mics_etc'] = pandas.to_datetime(
    music_df['after_first_digit'].dropna().loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None)].apply(
lambda x: x[re.search(date1,x).end():len(x)]), 
                   errors='coerce').dropna()

print('\nStep 5a. Canonical abbreviation and date1 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5b. Variant abbreviation and date1 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5c. Incorrect abbreviation and date1 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5d. Band not found in etree list date1 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''


music_df.loc[music_df['after_first_digit'].apply(
        lambda x: re.search(date2, x) != None), 
            'parsed_date'] = pandas.to_datetime(
    music_df['after_first_digit'][
    music_df['after_first_digit'].apply(
        lambda x: re.search(date2, x) != None)].apply(
lambda x: x[re.search(date2,x).start():re.search(date2,x).end()]), errors='coerce').dropna()

music_df.loc[music_df['after_first_digit'].apply(
        lambda x: re.search(date2, x) != None), 
            'parsed_date_mics_etc'] = music_df['after_first_digit'][
    music_df['after_first_digit'].apply(
        lambda x: re.search(date2, x) != None)].apply(
lambda x: x[re.search(date2,x).end():len(x)]).dropna()

music_df['parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'].dropna().loc[
    music_df['parsed_date_mics_etc'].dropna().apply(
    lambda x: re.search(markers, x) != None)].apply(
lambda x: x[re.search(markers, x).end():len(x)])



print('\nStep 5e. Canonical abbreviation and date2 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5f. Variant abbreviation and date2 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5g. Incorrect abbreviation and date2 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5h. Band not found in etree list date2 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''
music_df['parsed_date_mics_etc2'] = ''

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None), 'parsed_date'] = pandas.to_datetime(music_df['after_first_digit'].dropna()[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None)].apply(
    lambda x: re.sub('_','-',x[re.search(date3, x).start():re.search(date3, x).end()])), 
                   errors='coerce')

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None), 'parsed_date_mics_etc'] = music_df['after_first_digit'].dropna()[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None)].apply(
    lambda x: x[re.search(date3, x).end():len(x)])

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None), 
    'parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'][
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None)].apply(
lambda x: x[re.search(markers, x).end(): len(x)])

print('\nStep 5i. Canonical abbreviation and date3 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5j. Variant abbreviation and date3 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5k. Incorrect abbreviation and date3 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5l. Band not found in etree list date3 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''
music_df['parsed_date_mics_etc2'] = ''

music_df.loc[
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None), 
    'parsed_date'] = pandas.to_datetime(music_df['after_first_digit'][
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None)
].apply(
lambda x: x[re.search(date4, x).start():re.search(date4, x).end()]), errors='coerce')

music_df.loc[
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None), 
    'parsed_date_mics_etc'] = music_df['after_first_digit'][
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None)
].apply(
lambda x: x[re.search(date4, x).end():len(x)])

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date4, x) != None), 
    'parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'][
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date4, x) != None)].apply(
lambda x: x[re.search(markers, x).end(): len(x)])

print('\nStep 5m. Canonical abbreviation and date3 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5n. Variant abbreviation and date3 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5o. Incorrect abbreviation and date3 format: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5p. Band not found in etree list date3 format: score = 3')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


Step 5a. Canonical abbreviation and date1 format: score = 3
Filenames remaining in dataset: 548

Step 5b. Variant abbreviation and date1 format: score = 2
Filenames remaining in dataset: 548

Step 5c. Incorrect abbreviation and date1 format: score = 2
Filenames remaining in dataset: 545

Step 5d. Band not found in etree list date1 format: score = 3
Filenames remaining in dataset: 519

Step 5e. Canonical abbreviation and date2 format: score = 3
Filenames remaining in dataset: 513

Step 5f. Variant abbreviation and date2 format: score = 2
Filenames remaining in dataset: 511

Step 5g. Incorrect abbreviation and date2 format: score = 2
Filenames remaining in dataset: 507

Step 5h. Band not found in etree list date2 format: score = 3
Filenames remaining in dataset: 428

Step 5i. Canonical abbreviation and date3 format: score = 3
Filenames remaining in dataset: 428

Step 5j. Variant abbreviation and date3 format: score = 2
Filenames remaining in dataset: 428

Step 5k. Incorrect abbreviation

In [8]:
print('\nStep 6a. No band, date is yyyy-mm-dd: score = 2')
twos_df = twos_df.append(music_df.loc[(music_df['band'] == '') & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 6b. No band, date is yy-mm-dd: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''

music_df.loc[music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None), 
             'parsed_date'] = pandas.to_datetime(
    music_df['after_first_digit'].dropna().loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None)].apply(
lambda x: x[re.search(date1,x).start():re.search(date1,x).end()]), 
                   errors='coerce').dropna()

music_df.loc[music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None), 
             'parsed_date_mics_etc'] = pandas.to_datetime(
    music_df['after_first_digit'].dropna().loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date1, x) != None)].apply(
lambda x: x[re.search(date1,x).end():len(x)]), 
                   errors='coerce').dropna()

print('\nStep 6c. No band, date is date1 format: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''

music_df.loc[music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date2, x) != None), 
             'parsed_date'] = pandas.to_datetime(
    music_df['after_first_digit'].dropna().loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date2, x) != None)].apply(
lambda x: x[re.search(date2,x).start():re.search(date2,x).end()]), 
                   errors='coerce').dropna()

music_df.loc[music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date2, x) != None), 
             'parsed_date_mics_etc'] = pandas.to_datetime(
    music_df['after_first_digit'].dropna().loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date2, x) != None)].apply(
lambda x: x[re.search(date2,x).end():len(x)]), 
                   errors='coerce').dropna()

music_df['parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'].dropna().loc[
    music_df['parsed_date_mics_etc'].dropna().apply(
    lambda x: re.search(markers, x) != None)].apply(
lambda x: x[re.search(markers, x).end():len(x)])

print('\nStep 6d. No band, date is date2 format: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''
music_df['parsed_date_mics_etc2'] = ''

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None), 'parsed_date'] = pandas.to_datetime(music_df['after_first_digit'].dropna()[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None)].apply(
    lambda x: re.sub('_','-',x[re.search(date3, x).start():re.search(date3, x).end()])), 
                   errors='coerce')

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None), 'parsed_date_mics_etc'] = music_df['after_first_digit'].dropna()[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None)].apply(
    lambda x: x[re.search(date3, x).end():len(x)])

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None), 
    'parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'][
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date3, x) != None)].apply(
lambda x: x[re.search(markers, x).end(): len(x)])

print('\nStep 6e. No band, date is date3 format: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

music_df['parsed_date'] = np.nan
music_df['parsed_date_mics_etc'] = ''
music_df['parsed_date_mics_etc2'] = ''

music_df.loc[
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None), 
    'parsed_date'] = pandas.to_datetime(music_df['after_first_digit'][
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None)
].apply(
lambda x: x[re.search(date4, x).start():re.search(date4, x).end()]), errors='coerce')

music_df.loc[
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None), 
    'parsed_date_mics_etc'] = music_df['after_first_digit'][
    music_df['after_first_digit'].apply(lambda x: re.search(date4, x) != None)
].apply(
lambda x: x[re.search(date4, x).end():len(x)])

music_df.loc[
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date4, x) != None), 
    'parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'][
    music_df['after_first_digit'].dropna().apply(
    lambda x: re.search(date4, x) != None)].apply(
lambda x: x[re.search(markers, x).end(): len(x)])

print('\nStep 6f. No band, date is date4 format: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


Step 6a. No band, date is yyyy-mm-dd: score = 2
Filenames remaining in dataset: 377

Step 6b. No band, date is yy-mm-dd: score = 1
Filenames remaining in dataset: 377

Step 6c. No band, date is date1 format: score = 1
Filenames remaining in dataset: 340

Step 6d. No band, date is date2 format: score = 1
Filenames remaining in dataset: 335

Step 6e. No band, date is date3 format: score = 1
Filenames remaining in dataset: 335

Step 6f. No band, date is date4 format: score = 1
Filenames remaining in dataset: 335


In [82]:
music_df['date_by_hand'] = pandas.to_datetime(np.nan)
music_df['mics_etc'] = ''
#100
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 'date_by_hand'] = pandas.to_datetime('1997-06-22')
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 'mics_etc'] = 'beausolield1tr01'
#244
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'band'] = 'deadco'
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'mics_etc'] = 'irvine'
#281
music_df.loc[music_df['id'] == '587337.html', 'date_by_hand'] = pandas.to_datetime('2016-06-03')
music_df.loc[music_df['id'] == '587337.html', 'mics_etc'] = '01'
#282
music_df.loc[music_df['id'] == '587339.html', 'date_by_hand'] = pandas.to_datetime('2016-02-19')
music_df.loc[music_df['id'] == '587339.html', 'mics_etc'] = '01'
#390
music_df.loc[music_df['id'] == '587454.html', 'date_by_hand'] = pandas.to_datetime('2016-01-30')
music_df.loc[music_df['id'] == '587454.html', 'mics_etc'] = '01'
#492
music_df.loc[music_df['id'] == '587572.html', 'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '587572.html', 'mics_etc'] = '01'
#505
music_df.loc[music_df['id'] == '587588.html', 'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '587588.html', 'mics_etc'] = '01'
#523
music_df.loc[music_df['id'] == '587588.html', 'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '587588.html', 'mics_etc'] = '01'
#556
music_df.loc[music_df['id'] == '587588.html', 'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '587588.html', 'mics_etc'] = '01'
#686
music_df.loc[music_df['id'] == '587786.html', 'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '587786.html', 'mics_etc'] = 'george,wa t15 crosseyed and painless'
#781
music_df.loc[music_df['id'] == '587888.html', 'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '587888.html', 'band'] = 'phish-sleeptrain'
music_df.loc[music_df['id'] == '587888.html', 'mics_etc'] = 'cvca'
#881
music_df.loc[music_df['id'] == '565561.html', 'date_by_hand'] = pandas.to_datetime('2016-09-11')
music_df.loc[music_df['id'] == '565561.html', 'band'] = 'jimmy mo'
music_df.loc[music_df['id'] == '565561.html', 'mics_etc'] = 'mhile stor rte'
#955
music_df.loc[music_df['id'] == '565664.html', 'date_by_hand'] = pandas.to_datetime('2016-05-31')
music_df.loc[music_df['id'] == '565664.html', 'mics_etc'] = '210'
#1026
music_df.loc[music_df['id'] == '565762.html', 'date_by_hand'] = pandas.to_datetime('2016-07-03')
music_df.loc[music_df['id'] == '565762.html', 'mics_etc'] = '05'
#1037
music_df.loc[music_df['id'] == '565781.html', 'date_by_hand'] = pandas.to_datetime('2016-07-03')
music_df.loc[music_df['id'] == '565781.html', 'mics_etc'] = '05'
#1046
music_df.loc[music_df['id'] == '565793.html', 'date_by_hand'] = pandas.to_datetime('1981-06-27')
music_df.loc[music_df['id'] == '565793.html', 'mics_etc'] = '01'
#1115
music_df.loc[music_df['id'] == '565891.html', 'date_by_hand'] = pandas.to_datetime('1989-02-18')
music_df.loc[music_df['id'] == '565891.html', 'band'] = 'nickcave'
music_df.loc[music_df['id'] == '565891.html', 'mics_etc'] = 'd6_909_t01'
#1139
music_df.loc[music_df['id'] == '565923.html', 'date_by_hand'] = pandas.to_datetime('1997-07-05')
music_df.loc[music_df['id'] == '565923.html', 'band'] = 'vm'
music_df.loc[music_df['id'] == '565923.html', 'mics_etc'] = 'hamburg disc 02 track 02'
#1162
music_df.loc[music_df['id'] == '565954.html', 'date_by_hand'] = pandas.to_datetime('2016-04-29')
music_df.loc[music_df['id'] == '565954.html', 'band'] = 'ehg'
music_df.loc[music_df['id'] == '565954.html', 'mics_etc'] = 't01'
#1174
music_df.loc[music_df['id'] == '565970.html', 'date_by_hand'] = pandas.to_datetime('2016-06-06')
music_df.loc[music_df['id'] == '565970.html', 'band'] = 'the waterboys'
music_df.loc[music_df['id'] == '565970.html', 'mics_etc'] = 'destinies entwined'
#1245
music_df.loc[music_df['id'] == '566066.html', 'date_by_hand'] = pandas.to_datetime('2016-07-06')
music_df.loc[music_df['id'] == '566066.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566066.html', 'mics_etc'] = 't01'
#1270
music_df.loc[music_df['id'] == '566104.html', 'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566104.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566104.html', 'mics_etc'] = 't01'
#1277
music_df.loc[music_df['id'] == '566114.html', 'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566114.html', 'band'] = 'mavis staples'
music_df.loc[music_df['id'] == '566114.html', 'mics_etc'] = 't01'
#1288
music_df.loc[music_df['id'] == '566127.html', 'date_by_hand'] = pandas.to_datetime('1980-03-07')
music_df.loc[music_df['id'] == '566127.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566127.html', 'mics_etc'] = 'bolingbrook 16'
#1293
music_df.loc[music_df['id'] == '566132.html', 'date_by_hand'] = pandas.to_datetime('2016-07-06')
music_df.loc[music_df['id'] == '566132.html', 'mics_etc'] = '201'
#1305
music_df.loc[music_df['id'] == '566148.html', 'date_by_hand'] = pandas.to_datetime('2006-12-11')
music_df.loc[music_df['id'] == '566148.html', 'mics_etc'] = 'berlin_16_tr01'
#1314
music_df.loc[music_df['id'] == '566157.html', 'date_by_hand'] = pandas.to_datetime('1979-11-24')
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'the jam'
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'southampton_tr15'
#1356
music_df.loc[music_df['id'] == '566210.html', 'date_by_hand'] = pandas.to_datetime('2016-01-08')
#1365
music_df.loc[music_df['id'] == '566219.html', 'date_by_hand'] = pandas.to_datetime('1990-02-17')
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'pdp_loungeax'
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'd6_909_t01'
#1369
music_df.loc[music_df['id'] == '566223.html', 'date_by_hand'] = pandas.to_datetime('1988-07-09')
music_df.loc[music_df['id'] == '566157.html', 'band'] = ''
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = '07'
#1377
music_df.loc[music_df['id'] == '566234.html', 'date_by_hand'] = pandas.to_datetime('1979-11-18')
music_df.loc[music_df['id'] == '566234.html', 'band'] = 'jam'
music_df.loc[music_df['id'] == '566234.html', 'mics_etc'] = 'poole_15'
#1393
music_df.loc[music_df['id'] == '566252.html', 'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '566252.html', 'band'] = 'casting crowns'
music_df.loc[music_df['id'] == '566252.html', 'mics_etc'] = 'royals stadium kansas city missouri'
#1403
music_df.loc[music_df['id'] == '566264.html', 'date_by_hand'] = pandas.to_datetime('2016-07-10')
music_df.loc[music_df['id'] == '566264.html', 'band'] = 'paul mccartney'
music_df.loc[music_df['id'] == '566264.html', 'mics_etc'] = 't01'
#1449
music_df.loc[music_df['id'] == '566331.html', 'date_by_hand'] = pandas.to_datetime('2006-12-11')
music_df.loc[music_df['id'] == '566331.html', 'mics_etc'] = 'berlin_16_tr01'

#music_df['filename'].loc[1393]
print('ok')

ok


In [86]:
music_df[['id', 'filename', 'band', 'after_first_digit']][music_df.index > 1656]
#music_df['band'].loc[1656]

'wagner -'

In [29]:
music_df['year'] = -9999
music_df['mics_etc'] = ''
#185
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'year'] = 1995
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'mics_etc'] = 'd1t01.durango 95'
#219
music_df.loc[music_df['id'] == '8a7be6c05e405264de580672fbe6619a79955c43', 'year'] = 1975
music_df.loc[music_df['id'] == '8a7be6c05e405264de580672fbe6619a79955c43', 'mics_etc'] = 't01'
#224
music_df.loc[music_df['id'] == 'jg1975-xx-xx.boswell-smith.flac16', 'year'] = 1975
music_df.loc[music_df['id'] == 'jg1975-xx-xx.boswell-smith.flac16', 'mics_etc'] = 't01'
#271
music_df.loc[music_df['id'] == 'c23154fca319e57ed53f15a6a84e87a539312555', 'year'] = 1968
music_df.loc[music_df['id'] == 'c23154fca319e57ed53f15a6a84e87a539312555', 'mics_etc'] = 'los angeles,ca jonathan winters show mk1-tv-01'
#272
music_df.loc[music_df['id'] == '17a8a1fb659a7ccbc3df9d710ca2b1be9504c5c4', 'year'] = 1968
music_df.loc[music_df['id'] == '17a8a1fb659a7ccbc3df9d710ca2b1be9504c5c4', 'mics_etc'] = 'los angeles,ca jonathan winters show mk2-tv-01'
#388
music_df.loc[music_df['id'] == '587452.html', 'year'] = 1975
music_df.loc[music_df['id'] == '587452.html', 'month'] = 3
music_df.loc[music_df['id'] == '587452.html', 'mics_etc'] = 'xxt01'
#714
music_df.loc[music_df['id'] == '587815.html', 'year'] = 2016
music_df.loc[music_df['id'] == '587815.html', 'mics_etc'] = 't01'
#1223
music_df.loc[music_df['id'] == '565970.html', 'year'] = 1978
music_df.loc[music_df['id'] == '565970.html', 'mics_etc'] = 'gettysburg-tr01 entwined'
#1250
music_df.loc[music_df['id'] == '566072.html', 'year'] = 1985
music_df.loc[music_df['id'] == '566072.html', 'month'] = 10
music_df.loc[music_df['id'] == '566072.html', 'mics_etc'] = 'xxd01t01'
#1282
music_df.loc[music_df['id'] == '566119.html', 'year'] = 2016
music_df.loc[music_df['id'] == '566119.html', 'band'] = 'iggy pop'
music_df.loc[music_df['id'] == '566119.html', 'mics_etc'] = '01-isle of wight 2016.'
#1355
music_df.loc[music_df['id'] == '566208.html', 'year'] = 1972
music_df.loc[music_df['id'] == '566208.html', 'month'] = 1
music_df.loc[music_df['id'] == '566208.html', 'mics_etc'] = 'danish radio big band'
music_df.loc[music_df['id'] == '566208.html', 'mics_etc'] = 'k&atilde;&para;lner funkhaus'
#1375
music_df.loc[music_df['id'] == '566232.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566232.html', 'mics_etc'] = 'titel 01'
#1394
music_df.loc[music_df['id'] == '566253.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566232.html', 'mics_etc'] = '01 track0101'
#1460
music_df.loc[music_df['id'] == '566348.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566348.html', 'mics_etc'] = '01 all or nothing at all'
#1579
music_df.loc[music_df['id'] == '566506.html', 'year'] = 2016
music_df.loc[music_df['id'] == '566506.html', 'month'] = 7
music_df.loc[music_df['id'] == '566506.html', 'mics_etc'] = 'track01'
#1637
music_df.loc[music_df['id'] == '566581.html', 'year'] = 1998
music_df.loc[music_df['id'] == '566581.html', 'month'] = 1
music_df.loc[music_df['id'] == '566581.html', 'mics_etc'] = 'elvin'

'jefferson airplane 1968-xx-xx los angeles,ca jonathan winters show mk2-tv-01.flac'

In [ ]:
music_df['nodate'] = False
#87
music_df.loc[music_df['id'] == 'ddbaea0b694a4c0deaa8f75156446b8e7adea2d6', 'nodate'] = True
music_df.loc[music_df['id'] == 'ddbaea0b694a4c0deaa8f75156446b8e7adea2d6', 'mics_etc'] = '201'
#143
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'nodate'] = True
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'band'] = 'neil young'
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'mics_etc'] = 'd1t01 my my hey hey (out of the blue)'
#177
music_df.loc[music_df['id'] == '888ae778faf1d5ed69020113bf9f40806478f4ac', 'nodate'] = True
music_df.loc[music_df['id'] == '888ae778faf1d5ed69020113bf9f40806478f4ac', 'mics_etc'] = 't02.jack straw'
#183
music_df.loc[music_df['id'] == '52677374ccbd20cc35dbce25ac5b7018e9a8af0a', 'nodate'] = True
music_df.loc[music_df['id'] == '52677374ccbd20cc35dbce25ac5b7018e9a8af0a', 'band'] = ''
music_df.loc[music_df['id'] == '52677374ccbd20cc35dbce25ac5b7018e9a8af0a', 'mics_etc'] = 'd1t01'
#186
music_df.loc[music_df['id'] == 'edf25ec0ac4fd08874c7627bc527420091ef1747', 'nodate'] = True
music_df.loc[music_df['id'] == 'edf25ec0ac4fd08874c7627bc527420091ef1747', 'mics_etc'] = 'sl01_intro'
#192
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'nodate'] = True
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'mics_etc'] = 'rockaway beach'
#233
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'nodate'] = True
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'band'] = 'br'
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'mics_etc'] = '01'
#234
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'nodate'] = True
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'band'] = 'rush'
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'mics_etc'] = '1t01.-a passage to bangkok'
#239
music_df.loc[music_df['id'] == '1c5dc8fe10c17dcf42a786c638d6107939b81ef1', 'nodate'] = True
music_df.loc[music_df['id'] == '1c5dc8fe10c17dcf42a786c638d6107939b81ef1', 'mics_etc'] = '01 455 rocket'
#614
music_df.loc[music_df['id'] == '587704.html', 'nodate'] = True
music_df.loc[music_df['id'] == '587704.html', 'band'] = 'matrix'
music_df.loc[music_df['id'] == '587704.html', 'mics_etc'] = '01'
#651
music_df.loc[music_df['id'] == '587748.html', 'nodate'] = True
music_df.loc[music_df['id'] == '587748.html', 'band'] = 'matrix'
music_df.loc[music_df['id'] == '587748.html', 'mics_etc'] = '01'
#736
music_df.loc[music_df['id'] == '587838.html', 'nodate'] = True
music_df.loc[music_df['id'] == '587838.html', 'band'] = 't.flyer'
music_df.loc[music_df['id'] == '587838.html', 'mics_etc'] = 'disc01,track01'
#840
music_df.loc[music_df['id'] == '565503.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565503.html', 'band'] = 'the cure'
music_df.loc[music_df['id'] == '565503.html', 'mics_etc'] = 'le cure en montreal - 01 out of this world'
#870
music_df.loc[music_df['id'] == '565549.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565549.html', 'mics_etc'] = '1track01'
#879
music_df.loc[music_df['id'] == '565558.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565558.html', 'band'] = ''
music_df.loc[music_df['id'] == '565558.html', 'mics_etc'] = '1-01'
#902
music_df.loc[music_df['id'] == '565597.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565597.html', 'mics_etc'] = 'd1t01'
#910
music_df.loc[music_df['id'] == '565606.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565606.html', 'mics_etc'] = 'd1t01'
#919
music_df.loc[music_df['id'] == '565616.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565616.html', 'mics_etc'] = '210 - rosalita'
#939
music_df.loc[music_df['id'] == '565641.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565641.html', 'band'] = ''
music_df.loc[music_df['id'] == '565641.html', 'mics_etc'] = 'seattle2track01'
#948
music_df.loc[music_df['id'] == '565654.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565654.html', 'mics_etc'] = '01'
#950
music_df.loc[music_df['id'] == '565657.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565657.html', 'mics_etc'] = '205 atom heart mother'
#963
music_df.loc[music_df['id'] == '565675.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565675.html', 'mics_etc'] = '001_01'
#964
music_df.loc[music_df['id'] == '565676.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565676.html', 'mics_etc'] = 'sweet liberties'
music_df.loc[music_df['id'] == '565676.html', 'mics_etc'] = 'd101'
#970
music_df.loc[music_df['id'] == '565684.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565684.html', 'mics_etc'] = '08'
#977
music_df.loc[music_df['id'] == '565694.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565694.html', 'mics_etc'] = '01'
#982
music_df.loc[music_df['id'] == '565702.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565702.html', 'mics_etc'] = '1t01'
#993
music_df.loc[music_df['id'] == '565715.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565715.html', 'mics_etc'] = '101 the motel'
#994
music_df.loc[music_df['id'] == '565716.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565716.html', 'mics_etc'] = '209 light of day'
#1007
music_df.loc[music_df['id'] == '565738.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565738.html', 'band'] = ''
music_df.loc[music_df['id'] == '565738.html', 'mics_etc'] = '209 light of day'
#1013
music_df.loc[music_df['id'] == '565738.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565738.html', 'mics_etc'] = 'on fire'
#1027
music_df.loc[music_df['id'] == '565763.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565763.html', 'band'] = ''
music_df.loc[music_df['id'] == '565763.html', 'mics_etc'] = 't01 - intro - land of hope &amp; glory - over the rainbow'
#1028
music_df.loc[music_df['id'] == '565738.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565738.html', 'mics_etc'] = '01'
#1030
music_df.loc[music_df['id'] == '565763.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565763.html', 'band'] = ''
music_df.loc[music_df['id'] == '565763.html', 'mics_etc'] = 'd1t01'
#1056
music_df.loc[music_df['id'] == '565804.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565804.html', 'mics_etc'] = '101 intro'
#1069
music_df.loc[music_df['id'] == '565826.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565826.html', 'band'] = ''
music_df.loc[music_df['id'] == '565826.html', 'mics_etc'] = 'r09_0004_r8b01'
#1082
music_df.loc[music_df['id'] == '565844.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565844.html', 'mics_etc'] = '01 hold on i\'m coming'
#1094
music_df.loc[music_df['id'] == '565863.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565863.html', 'band'] = ''
music_df.loc[music_df['id'] == '565863.html', 'mics_etc'] = 'd1t01'
#1119
music_df.loc[music_df['id'] == '565896.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565896.html', 'mics_etc'] = '01'
#1124
music_df.loc[music_df['id'] == '565901.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565901.html', 'band'] = 'whitford'
music_df.loc[music_df['id'] == '565901.html', 'mics_etc'] = 'st01'
#1132
music_df.loc[music_df['id'] == '565911.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565911.html', 'mics_etc'] = 'st01'
#1135
music_df.loc[music_df['id'] == '565915.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565915.html', 'mics_etc'] = 'd1t01'
#1138
music_df.loc[music_df['id'] == '565922.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565922.html', 'mics_etc'] = '01 kyrie aus der messe in h-moll 01-36'
#1145
music_df.loc[music_df['id'] == '565929.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565929.html', 'mics_etc'] = '01 prometheus 06-31'
#1151
music_df.loc[music_df['id'] == '565938.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565938.html', 'mics_etc'] = '07'
#1172
music_df.loc[music_df['id'] == '565966.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565966.html', 'mics_etc'] = '01 tribute to keith jarrett 11-39'
#1181
music_df.loc[music_df['id'] == '565981.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565981.html', 'mics_etc'] = '01 scollin\' 08-29'
#1194
music_df.loc[music_df['id'] == '565997.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565997.html', 'band'] = ''
music_df.loc[music_df['id'] == '565997.html', 'mics_etc'] = 'a01'
#1200
music_df.loc[music_df['id'] == '566005.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566005.html', 'mics_etc'] = '101 mr. tambourine man'
#1213
music_df.loc[music_df['id'] == '566020.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566020.html', 'mics_etc'] = 'd2t06 - biko'
#1230
music_df.loc[music_df['id'] == '566046.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566046.html', 'mics_etc'] = '101 - [venue announcements]'
#1233
music_df.loc[music_df['id'] == '566050.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566050.html', 'band'] = 'black sabbath'
music_df.loc[music_df['id'] == '566050.html', 'mics_etc'] = 'a01'
#1316
music_df.loc[music_df['id'] == '566159.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566159.html', 'mics_etc'] = '01 shady grove'
#1326
music_df.loc[music_df['id'] == '566170.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566170.html', 'mics_etc'] = '01 1'
#1328
music_df.loc[music_df['id'] == '566175.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566175.html', 'mics_etc'] = '01'
#1331
music_df.loc[music_df['id'] == '566178.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566178.html', 'mics_etc'] = '00'
#1334
music_df.loc[music_df['id'] == '566181.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566181.html', 'band'] = ''
music_df.loc[music_df['id'] == '566181.html', 'mics_etc'] = 't01 after the goldrush'
#1354
music_df.loc[music_df['id'] == '566207.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566207.html', 'band'] = ''
music_df.loc[music_df['id'] == '566207.html', 'mics_etc'] = 'd1t01'
#1361
music_df.loc[music_df['id'] == '566215.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566215.html', 'mics_etc'] = '01 little black dress'
#1394
music_df.loc[music_df['id'] == '566253.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566253.html', 'mics_etc'] = '01 track0101'
#1404
music_df.loc[music_df['id'] == '566266.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566266.html', 'mics_etc'] = '01 ode to satie 11-24'
#1416
music_df.loc[music_df['id'] == '566280.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566280.html', 'band'] = 'thunderstone'
music_df.loc[music_df['id'] == '566280.html', 'mics_etc'] = 'soundcheck'
#1439
music_df.loc[music_df['id'] == '566317.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566317.html', 'mics_etc'] = '1-01 rainy day women #12 &amp; 35'
#1441
music_df.loc[music_df['id'] == '566320.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566320.html', 'mics_etc'] = '01'
#1476
music_df.loc[music_df['id'] == '566369.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566369.html', 'mics_etc'] = '208-light of day'
#1483
music_df.loc[music_df['id'] == '566377.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566377.html', 'mics_etc'] = 'marque 01'
#1491
music_df.loc[music_df['id'] == '566377.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566377.html', 'mics_etc'] = 'das rheingold part 1'
#1512
music_df.loc[music_df['id'] == '566417.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566417.html', 'mics_etc'] = '01 interiors 13-38'
#1530
music_df.loc[music_df['id'] == '566436.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566436.html', 'mics_etc'] = '204 mary\'s place'
#1533
music_df.loc[music_df['id'] == '566439.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566439.html', 'mics_etc'] = 'hw'
music_df.loc[music_df['id'] == '566439.html', 'mics_etc'] = 'toads place matrix _01'
#1535
music_df.loc[music_df['id'] == '566445.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566445.html', 'mics_etc'] = '101 - 3 stooges intro'
#1539
music_df.loc[music_df['id'] == '566451.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566451.html', 'mics_etc'] = '01 hot tamale baby'
#1550
music_df.loc[music_df['id'] == '566465.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566465.html', 'mics_etc'] = '01 - die walkure act 1'
#1562
music_df.loc[music_df['id'] == '566485.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566485.html', 'mics_etc'] = '01 dori theme 06-35'
#1567
music_df.loc[music_df['id'] == '566491.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566491.html', 'mics_etc'] = 't01'
#1568
music_df.loc[music_df['id'] == '566492.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566492.html', 'mics_etc'] = '01 unknown'
#1595
music_df.loc[music_df['id'] == '566525.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566525.html', 'mics_etc'] = '02 - 01-stratovarius'
#1606
music_df.loc[music_df['id'] == '566539.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566539.html', 'mics_etc'] = ' 01 - siegfried act 1'
#1622
music_df.loc[music_df['id'] == '566560.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566560.html', 'mics_etc'] = '22'
#1627
music_df.loc[music_df['id'] == '566568.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566568.html', 'mics_etc'] = 'd1track01'
#1631
music_df.loc[music_df['id'] == '566572.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566572.html', 'mics_etc'] = '06'
#1636
music_df.loc[music_df['id'] == '566580.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566580.html', 'mics_etc'] = '101 save it for me'
#1641
music_df.loc[music_df['id'] == '566594.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566594.html', 'mics_etc'] = 'kv01'
#1643
music_df.loc[music_df['id'] == '566598.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566598.html', 'band'] = ''
music_df.loc[music_df['id'] == '566598.html', 'mics_etc'] = 'd01'
#1644
music_df.loc[music_df['id'] == '566604.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566604.html', 'band'] = ''
music_df.loc[music_df['id'] == '566604.html', 'mics_etc'] = 'n01'
#1645
music_df.loc[music_df['id'] == '566607.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566607.html', 'band'] = ''
music_df.loc[music_df['id'] == '566607.html', 'mics_etc'] = 'tt01'
#1656
music_df.loc[music_df['id'] == '566621.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566621.html', 'mics_etc'] = '01 - g&atilde;&para;tterd&atilde;&curren;mmerung act 1 part 1'

In [ ]:

print('\nStep 9a. No date, no band: score = 0')
zeros_df = zeros_df.append(
    music_df.loc[
        (music_df['band'] == '') & 
        (music_df['nodate'])
        ])
music_df = music_df.loc[~music_df.index.isin(zeros_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 9b. No date, band found: score = 1')
ones_df = ones_df.append(music_df.loc[music_df['nodate']])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))



In [ ]:


music_df[['id', 'filename', 'band', 'after_first_digit', 'mics_etc']][music_df.index > 1430]



In [ ]:

# REMOVE TRAILING CHARACTERS FROM BAND NAME
'''
print('/'+music_df['band'].loc[2762]+'/')

# Parse away markers (e.g. _) off beginning and end of band name
music_df.loc[
    music_df['band'].apply(
        lambda x: re.search(endmarkers, x) != None)
, 'band'] = music_df['band'][
    music_df['band'].apply(
        lambda x: re.search(endmarkers, x) != None)
].apply(
    lambda x: 
    x[0:re.search(endmarkers, x).start()].lower().strip()
)
'''


In [ ]:

# Check band names vs. list of official abbreviations
canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()
music_df['canonical_abbrev'] = False
music_df.loc[music_df['band'].isin(canonical_abbrevs), 'canonical_abbrev'] = True


# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)

music_df['variant_abbrev'] = False
music_df.loc[music_df['band'].isin(variant_abbrevs), 'variant_abbrev'] = True

# Now parse dates. First, try to parse as correctly-formatted yyyy-mm-dd date
music_df['thedate'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = ''
music_df.loc[music_df['thedate'].notnull(), 'mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[10+re.search('[\.\_\-\ ]*',x[10:]).end():])

# STEP 2: Find correct abbreviations and yyyy-mm-dd dates, assign score = 4
if (debug > 0):
    print('\n')
    print('Step 2: Band has canonical abbreviation and date is correctly yyyy-mm-dd.')

fours = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'thedate', 'mics_etc']][(music_df['canonical_abbrev']) & (music_df['thedate'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['canonical_abbrev']) & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

    
# STEP 3: Find variant abbreviations and yyyy-mm-dd dates, assign score = 3
if (debug > 0):
    print('\n')
    print('Step 3: Band has variant abbreviation and date is correctly yyyy-mm-dd.')

music_df.loc[(music_df['variant_abbrev']) & (music_df['thedate'].notnull()), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'thedate', 'mics_etc']][(music_df['variant_abbrev']) & (music_df['thedate'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['variant_abbrev']) & (music_df['thedate'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

In [ ]:

music_df['band'] = music_df['filename'].apply(lambda x: x[0:re.search('\d', x).start()])
music_df['after_first_digit'] = music_df['filename'].apply(lambda x: x[re.search('\d', x).start():len(x)])


zeros_df = zeros_df.append(music_df.loc[music_df['filename'].apply(lambda x: re.search('^(disc|cd|track)\d', x.lower()) != None)])

music_df = music_df[~music_df.isin(zeros_df)].dropna()
print(len(music_df))


# regexes that indicate a filename should be scored as zero:
# '^(disc|cd|track)\d' , 
music_df['filename'].sample(30)
music_df['filename'][(music_df['filename'].apply(lambda x: re.search('^0\d', x.lower()) != None)) & (music_df['filename'].apply(lambda x: len(x)<=30))].sort_values().tolist()


In [ ]:

# Initialize score to -9999
music_df['score'] = -9999

# Create outmusic_df as the place to move data to when a score has been assigned
thecolumns = music_df.columns
outmusic_df = pandas.DataFrame(data=None, columns=thecolumns) # Trick to copy schema but not data
outmusic_df.index.name = music_df.index.name

# All filenames have a period somewhere
#music_df[music_df['filename'].apply(lambda x: '.' not in x)]

# First, strip filename and convert to lowercase.
music_df['filename'] = music_df['filename'].apply(lambda x: x.lower().strip())

# Next, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])


# STEP 1: If there are no numbers at all, there's no date, so score as zero
if (debug > 0):
    print('Step 1: filter out filenames with no numbers, which cannot possibly be correct.')
music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score']][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Get band abbrev by parsing up to the first digit
music_df['band'] = music_df['remainder'].apply(lambda x: x[0:re.search('\d', x).start()].lower().strip())

# Fix U2
music_df.loc[music_df['band'] == 'u', 'band'] = 'u2'

# Parse everything after the first digit for analysis of dates
music_df['after_first_digit'] = music_df['remainder'].apply(lambda x: x[re.search('\d', x).start():len(x)].lower().strip())




In [ ]:
music_df['thedate'] = pandas.to_datetime(np.nan)

# STEP 4: If it has the word "track" in the filename, it doesn't have band or date, score = 0
if (debug > 0):
    print('\n')
    print('Step 4: The word "track" appears, score zero (checked that this is correct.)')

music_df.loc[music_df['band'].apply(lambda x: 'track' in x), 'score'] = 0
outmusic_df = outmusic_df.append(
    music_df[
        ['id', 'source', 'filename', 'filetype', 
         'score', 'band', 'thedate', 'mics_etc']
    ][(music_df['band'].apply(lambda x: 'track' in x)

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['band'].apply(lambda x: 'track' in x)])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


In [ ]:




# STEP 3: Correct abbreviation and INCORRECT date as yy-mm-dd
if (debug > 0):
    print('\n')
    print('Step 3: Band has canonical abbreviation and date is incorrectly yy-mm-dd.')

# Next, try to parse dates as yy-mm-dd
music_df['thedate'] = pandas.to_datetime(np.nan)
music_df['thedate'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df.loc[music_df['thedate'].notnull(), 'mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[8+re.search('[\.\_\-\ ]*',x[8:]).end():])

music_df.loc[(music_df['canonical_abbrev']) & (music_df['thedate'].notnull()), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'thedate', 'mics_etc']][(music_df['canonical_abbrev']) & (music_df['thedate'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['canonical_abbrev']) & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


    

# STEP 4: Find variant abbreviations and yyyy-mm-dd dates, assign score = 3
if (debug > 0):
    print('\n')
    print('Step 4: Band has variant abbreviation and date is correctly yyyy-mm-dd.')

    
    
music_df['variant_abbrev'] = False
music_df.loc[music_df['band'].isin(variant_abbrevs), 'variant_abbrev'] = True

music_df['thedate'] = pandas.to_datetime(np.nan)
music_df['thedate'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')



music_df.loc[(music_df['variant_abbrev']) & (music_df['thedate'].notnull()), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'thedate', 'mics_etc']][(music_df['canonical_abbrev']) & (music_df['thedate'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['canonical_abbrev']) & (music_df['thedate'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


neworder = ['score', 'band', 'thedate', 'mics_etc', 'filetype', 'source', 'filename', 'id']
outmusic_df = outmusic_df[neworder]
outmusic_df = outmusic_df.fillna('')

#outmusic_df.to_csv('outmusic.csv')
#print('\n')
#print('File written')

In [ ]:
outmusic_df

In [ ]:







fdy = re.compile('(19|20)\d\d[\ \-\_]+\d+[\ \-\_]+\d+')

music_df.loc[(music_df['full_date'].isnull()) & 
             (music_df['yy_date'].isnull()) &
             (music_df['parsed_date'].isnull()) &
             (music_df['after_first_digit'].apply(lambda x: re.search(fdy, x) != None)), 
         'parsed_date'
        ] = pandas.to_datetime(
    music_df['filename'][
        (music_df['full_date'].isnull()) & 
        (music_df['parsed_date'].isnull()) &
        (music_df['after_first_digit'].apply(lambda x: re.search(fdy, x) != None))
        ].apply(lambda x: x[re.search(fdy, x).start():re.search(fdy, x).end()])
    , errors='coerce'
)

In [ ]:

            
#music_df['variant_abbrev'] = False
#music_df.loc[music_df['band'].isin(variant_abbrevs), 'variant_abbrev'] = True
#variant_abbrevs
#

In [ ]:

#music_df['remainder'][music_df['band'] == 'u2']

In [ ]:

# STEP 3: Band has canonical abbreviation and date is yy-mm-dd
if (debug > 0):
    print('\n')
    print('Step 3: Band has canonical abbreviation and date is yy-mm-dd.')
music_df.loc[(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull()), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'yy_date', 'mics_etc']][(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
#music_df[(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull())]


In [ ]:
debug = 1
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
 
# Initialize score to -9999
music_df['score'] = -9999

# Create outmusic_df as the place to move data to when a score has been assigned
thecolumns = music_df.columns
outmusic_df = pandas.DataFrame(data=None, columns=thecolumns) # Trick to copy schema but not data
outmusic_df.index.name = music_df.index.name

# All filenames have a period somewhere
#music_df[music_df['filename'].apply(lambda x: '.' not in x)]

# First, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])

#print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# STEP 1: If there are no numbers at all, there's no date, so score as zero
if (debug > 0):
    print('Step 1: filter out filenames with no numbers, which cannot possibly be correct.')
music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score']][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# STEP 2: DOES THE BAND ABBREVIATION MATCH AN OFFICIAL ABBREVIATION?

# STEP 2a: Parse out the band name and check against the abbreviations list
# first, parse to first number - that's where the date starts (usually, but not for U2 or 4 Non-Blondes)
music_df['band_maybe'] = music_df['remainder'].apply(lambda x: x.lower()[0:re.search('\d', x).start()])
music_df['abbrev_ok'] = False
music_df.loc[music_df['band_maybe'].isin(abbrev_df['abbrev'][abbrev_df['abbrev'] != '']), 'abbrev_ok'] = True
# abbrev_ok = True: 296

# Step 2b: If abbrev_ok, cut band name off front of remainder
music_df.loc[music_df['abbrev_ok'], 'remainder'] = music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[re.search('\d', x).start():])

# Step 2c: Look for proper datetime format of yyyy-mm-dd and assign score of 4
if (debug > 0):
    print('\n')
    print('Step 2c: Band abbreviation from official list plus date in correct yyyy-mm-dd format: score as 4.')
music_df['thedate'] = pandas.to_datetime(music_df[music_df['abbrev_ok']]['remainder'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 4
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 4])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 2d: Format yy-mm-dd gets score of 3
if (debug > 0):
    print('\n')
    print('Step 2d: Band abbreviation from official list plus date in incorrect yy-mm-dd format: score as 3.')
music_df['thedate'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):    
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 2e: Three clusters of numbers or xx, optionally with - or / separator, is a date.
if (debug > 0):
    print('\n')
    print('Step 2e: Band abbreviation from official list plus date in other incorrect format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['abbrev_ok']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
music_df['mics_etc'] = music_df['remainder'][(music_df['abbrev_ok']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 2f: No date provided but band abbrev OK, score as 3
if (debug > 0):
    print('\n')
    print('Step 2f: Band abbreviation from official list plus no date provided: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df.loc[(music_df['abbrev_ok']), 'score'] = 3
music_df.loc[music_df['abbrev_ok'], 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[music_df['abbrev_ok'], 'mics_etc'] = music_df['remainder']
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])    
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


# STEP 3: DOES THE BAND ABBREVIATION MATCH AN VARIANT ABBREVIATION? MODIFY AND REPEAT STEPS (a) through (f) above

# strip whitespace from band name and put as lowercase for easier processing
music_df['band_maybe'] = music_df['band_maybe'].apply(lambda x: x.lower().strip())

# Make a list of non-canonical abbreviations to check
otherabraw = abbrev_df['otherabbrev'][abbrev_df['otherabbrev'].apply(lambda x: x not in ['',' '])].drop_duplicates().values.tolist()

# Parse comma-separated values in non-canonical abbreviations list as separate possibilities; also strip whitespace
otherabbrevs = []
for x in otherabraw:
    if (',' in x):
        for y in x.split(','):
            otherabbrevs.append(y.strip())
    else:
        otherabbrevs.append(x.strip())

# If band name is a match to that non-canonical abbrev list, set abbrev_variant = True
music_df['abbrev_variant'] = False
music_df.loc[music_df['band_maybe'].isin(otherabbrevs), 'abbrev_variant'] = True
#print(len(music_df[music_df['abbrev_variant']]))
# abbrev_variant = True: 40

# Step 3b: If abbrev_variant, cut band name off front of remainder
music_df.loc[music_df['abbrev_variant'], 'remainder'] = music_df['remainder'][music_df['abbrev_variant']].apply(lambda x: x[re.search('\d', x).start():])

# Step 3c: Look for proper datetime format of yyyy-mm-dd; with the variant band name assign score of 3
if (debug > 0):
    print('\n')
    print('Step 3c: Band abbreviation from variant list plus date in correct yyyy-mm-dd format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df[music_df['abbrev_variant']]['remainder'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 3d: Format yy-mm-dd plus variant band name gets score of 2
if (debug > 0):
    print('\n')
    print('Step 3d: Band abbreviation from variant list plus date in incorrect yy-mm-dd format: score as 2.')

music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['abbrev_variant']].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 2
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 2]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 2])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 3e: Three clusters of numbers or xx, optionally with - or / separator, is a date; plus variant band name, score as 2
if (debug > 0):
    print('\n')
    print('Step 3e: Band abbreviation from variant list plus date in other incorrect format: score as 2.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['abbrev_variant']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
music_df['mics_etc'] = music_df['remainder'][(music_df['abbrev_variant']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 2
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 2]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 2])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 3f: No date provided but band abbrev is a variant, score as 2
if (debug > 0):
    print('\n')
    print('Step 3f: Band abbreviation from variant list plus no date provided: score as 2.')

music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df.loc[(music_df['abbrev_variant']), 'score'] = 2
music_df.loc[music_df['abbrev_variant'], 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[music_df['abbrev_variant'], 'mics_etc'] = music_df['remainder']
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 2]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 2])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

    
# Step 4: If they have the word "track" in the title, gets scored as zero (confirmed by eye)
#music_df['filename'][music_df['track_as_band']].sort_values().tolist()
if (debug > 0):
    print('\n')
    print('Step 4: The word "track" is present in the title: score as 0 (confirmed by eye as correct approach).')
music_df['track_as_band'] = False
music_df.loc[music_df['band_maybe'].apply(lambda x: 'track' in x), 'track_as_band'] = True
music_df.loc[music_df['track_as_band'], 'band_maybe'] = ''
music_df.loc[music_df['track_as_band'], 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[music_df['track_as_band'], 'mics_etc'] = music_df['remainder']
music_df.loc[(music_df['track_as_band']), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 0]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 0])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))    
    
# STEP 5a: Apply fix for U2
music_df.loc[music_df['band_maybe'].apply(lambda x: x.lower()) == 'u', 'band_maybe'] = 'U2'

# Step 5b: Fix remainder for U2
music_df.loc[music_df['band_maybe'] == 'U2', 'remainder'] = music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[re.search('u2[\ \-\_]*',x.lower()).end():len(x)])

#print('ok')
if (debug == 2):
    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'u'])
    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'U2'])

# Step 5c: band is U2 (no official abbrev) and date is yyyy-mm-dd, score as 4.
if (debug > 0):
    print('\n')
    print('Step 5c: Band is U2 plus date in correct yyyy-mm-dd format: score as 4.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x.lower()[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 4
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 4])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5d: Band is U2, format yy-mm-dd gets score of 3
if (debug > 0):
    print('\n')
    print('Step 5d: Band is U2 plus date in incorrect yy-mm-dd format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''

music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5e: Three clusters of numbers or xx, optionally with - or / separator, is a date; plus band is U2, score as 3
if (debug > 0):
    print('\n')
    print('Step 5e: Band is U2 plus date in other incorrect format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
music_df['mics_etc'] = music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 5f: No date provided but band is U2, score as 3
if (debug > 0):
    print('\n')
    print('Step 5f: Band is U2 plus no date provided: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df.loc[(music_df['band_maybe'] == 'U2'), 'score'] = 3
music_df.loc[(music_df['band_maybe'] == 'U2'), 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[(music_df['band_maybe'] == 'U2'), 'mics_etc'] = music_df['remainder']
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))    


#outmusic_df.to_csv('outmusic.csv')
#print('File written')

print('\n')
print('Three rows from the output data, selected at random:')
outmusic_df.sample(3)

In [ ]:
# Band provided but not processed from abbrevs list
#print(len(music_df))

# I think these are trying to provide track number instead of band name
badformat = ['d', 't', 'a']

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic', 'jg+ms', 'deadco', 'deadandco', 'dc', 'jgms', 'ph']
incorrect += ['dead and company', 'dead.and.co', 'dead.and.co', 'dac', 'dandc']
incorrect += ['deadco-']


# Band names not in abbreviation list
notfound = ['loslobos', 'deanmonkey&amp;thedropouts', 'ttb', 'crb', 'deadandcompany', 'ny', 'gsbg', 'bd']
notfound += ['jefferson starship', 'carlton, larry -', 'the who', 's marriott', 'unknown artist']
notfound += ['tal wilkenfeld', 'ph&amp;tl', 'sb', 'radiohead', 'theclash_', 'primus']

vc = music_df['band_maybe'][music_df['band_maybe'] != ''].value_counts()
# If a band name appears 4 or fewer times, assume it's a misc band not found in the list.
for x in (vc[(vc<5)].index.sort_values().tolist()):
    if ((x not in notfound) and (x not in incorrect) and (x not in badformat)):
        notfound.append(x)
    

# STEP 6a: Used incorrect band name
music_df['remainder'][music_df['band_maybe'].apply(lambda x: x in incorrect)].apply(lambda x: x[0:19])


music_df.loc[music_df['abbrev_ok'], 'remainder'] = music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[re.search('\d', x).start():])


# Step 6b: Set remainder by Parsing to first digit


#music_df.loc[music_df['band_maybe'].apply(lambda x: x.lower()) == 'u', 'band_maybe'] = 'U2'

# Step 5b: Fix remainder for U2
#music_df.loc[music_df['band_maybe'] == 'U2', 'remainder'] = music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[re.search('u2[\ \-\_]*',x.lower()).end():len(x)])

#print('ok')
#if (debug == 2):
#    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'u'])
#    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'U2'])

# Step 5c: band is U2 (no official abbrev) and date is yyyy-mm-dd, score as 4.
#if (debug > 0):
#    print('\n')
#    print('Step 5c: Band is U2 plus date in correct yyyy-mm-dd format: score as 4.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''
#music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x.lower()[0:10]), format='%Y-%m-%d', errors='coerce')
#music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
#music_df.loc[music_df['thedate'].notnull(), 'score'] = 4
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 4])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5d: Band is U2, format yy-mm-dd gets score of 3
#if (debug > 0):
#    print('\n')
#    print('Step 5d: Band is U2 plus date in incorrect yy-mm-dd format: score as 3.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''

#music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
#music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
#music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 3])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5e: Three clusters of numbers or xx, optionally with - or / separator, is a date; plus band is U2, score as 3
#if (debug > 0):
#    print('\n')
#    print('Step 5e: Band is U2 plus date in other incorrect format: score as 3.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''
#music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
#music_df['mics_etc'] = music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
#music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 3])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 5f: No date provided but band is U2, score as 3
#if (debug > 0):
#    print('\n')
#    print('Step 5f: Band is U2 plus no date provided: score as 3.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''
#music_df.loc[(music_df['band_maybe'] == 'U2'), 'score'] = 3
#music_df.loc[(music_df['band_maybe'] == 'U2'), 'thedate'] = pandas.to_datetime(np.nan)
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 3])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))    

#neworder = ['score', 'band_maybe', 'thedate', 'mics_etc', 'filetype', 'source', 'filename', 'id']
#outmusic_df = outmusic_df[neworder]
#outmusic_df = outmusic_df.rename(columns={'band_maybe': 'band'})
#
##outmusic_df.to_csv('outmusic.csv')
##print('File written')

#print('\n')
#print('Three rows from the output data, selected at random:')
#outmusic_df.sample(3)

#print(music_df[music_df['band_maybe'] != ''].groupby(['band_maybe']).size().sort_values(ascending=False))


#vc[(vc==1)].sort_index().index.tolist()
#music_df[music_df['band_maybe'] == 'dm']
#print(notfound)

In [ ]:
sorted(notfound)